# Fragestellungen / Hypothesen

- **H1**: Wie verändert sich die Herzfrequenz beim Joggen mit der Änderung des Intensitätslevel?

- **H2**: Welchen Einfluss hat das Fitnesslevel auf den BPM beim Joggen verschiedener Intensitätslevels?

- **H3**: Hat der Raucherstatus Einfluss auf die Herzfrequenz beim Joggen verschiedener Intensitätslevels?

# Methodisches Vorgehen

**Datengrundlage:**  
- Messgerät: Polar H10 Pulsgurt, Datenexport via Elite HRV App  
- Variablen: BPM (bereinigt), RR-Intervalle, Intensitätslevel (1=niedrig, 2=mittel, 3=hoch)
- Metadaten: Alter, Geschlecht, Raucherstatus, Fitnesslevel, BMI, Größe

**Datenaufbereitung:**  
- Berechnung BPM aus RR-Intervallen (60000 / RR_ms)
- Bereinigung mittels Median-Filter (Kernel-Size 5-11)
- Filterung unrealistischer Werte (BPM < 40 oder > 200)
- Zusammenführung mit Metadaten (Alter, Geschlecht, Fitnesslevel, etc.)

**Analyseansatz:**  
- Visualisierungen: Boxplots BPM nach Intensitätslevel, Violinplots, Interaktionsplots
- Deskriptive Statistik: Mittelwert, Median, Standardabweichung, Verteilungen
- Hypothesentests: ANOVA, zweifaktorielle ANOVA, Voraussetzungsprüfungen

In [12]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import shapiro, levene, f_oneway
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Load Data using DataLoader class
data_path = "data/clean"

class DataLoader:
    def __init__(self, data_path):
        self.data_path = data_path
    
    def load_all_cleaned_data(self):
        """Load all cleaned activity data files"""
        all_dfs = []
        for root, dirs, files in os.walk(self.data_path):
            for file in files:
                if file.endswith('_activity_lsdlpr25.csv'):
                    file_path = os.path.join(root, file)
                    df = pd.read_csv(file_path)
                    all_dfs.append(df)
        return pd.concat(all_dfs, ignore_index=True)

# Load data
data_loader = DataLoader(data_path)
df = data_loader.load_all_cleaned_data()

# Create categorical labels
df['intensity_label'] = df['intensity'].map({1: 'niedrig', 2: 'mittel', 3: 'hoch'})
df['fitness_label'] = df['fitness_level'].map({0: 'niedrig', 1: 'mittel', 2: 'hoch'})
df['smoker_label'] = df['smoker'].map({0: 'Nichtraucher', 1: 'Raucher'})

print(f"Total records: {len(df)}")
print(f"Unique persons: {df['person_id'].nunique()}")
print(f"\nData overview:")
display(df[['person_id', 'age', 'fitness_label', 'smoker_label', 'intensity_label', 'bpm_clean']].head(10))

Total records: 3178
Unique persons: 3

Data overview:


,person_id,age,fitness_label,smoker_label,intensity_label,bpm_clean
0,3,26,mittel,Nichtraucher,niedrig,108.524945
1,3,26,mittel,Nichtraucher,niedrig,87.209302
2,3,26,mittel,Nichtraucher,niedrig,90.553008
3,3,26,mittel,Nichtraucher,niedrig,93.896714
4,3,26,mittel,Nichtraucher,niedrig,93.896714
5,3,26,mittel,Nichtraucher,niedrig,93.896714
6,3,26,mittel,Nichtraucher,niedrig,98.039216
7,3,26,mittel,Nichtraucher,niedrig,98.039216
8,3,26,mittel,Nichtraucher,niedrig,104.895105
9,3,26,mittel,Nichtraucher,niedrig,105.820106


# H1: Zusammenhang zwischen Intensitätslevel und Herzfrequenz (BPM)

## Visualisierungen

In [14]:
# Boxplot der Herzfrequenz nach Intensitätslevel
fig1 = px.box(df, x='intensity_label', y='bpm_clean',
              title='Herzfrequenz (BPM) nach Intensitätslevel',
              labels={'intensity_label': 'Intensitätslevel', 'bpm_clean': 'Herzfrequenz (BPM)'},
              color='intensity_label',
              category_orders={'intensity_label': ['niedrig', 'mittel', 'hoch']})
fig1.update_layout(showlegend=False)
fig1.show()

# Violinplot zur Verteilung
fig2 = px.violin(df, x='intensity_label', y='bpm_clean',
                 title='Verteilung der Herzfrequenz nach Intensitätslevel',
                 labels={'intensity_label': 'Intensitätslevel', 'bpm_clean': 'Herzfrequenz (BPM)'},
                 color='intensity_label',
                 box=True,
                 category_orders={'intensity_label': ['niedrig', 'mittel', 'hoch']})
fig2.update_layout(showlegend=False)
fig2.show()

## Deskriptive Statistik

In [15]:
# Berechnung von Mittelwert, Median und Standardabweichung je Intensitätslevel
h1_stats = df.groupby('intensity_label')['bpm_clean'].agg([
    ('Mittelwert', 'mean'),
    ('Median', 'median'),
    ('Standardabweichung', 'std'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('n', 'count')
]).round(2)

print("=== H1: Deskriptive Statistik nach Intensitätslevel ===")
display(h1_stats)

=== H1: Deskriptive Statistik nach Intensitätslevel ===


,Mittelwert,Median,Standardabweichung,Min,Max,n
intensity_label,,,,,,
hoch,175.14,180.18,16.72,65.36,191.69,1180
mittel,160.74,163.49,16.21,89.42,182.93,1022
niedrig,139.87,141.51,14.18,87.21,163.04,976


## Hypothesentests

In [16]:
# ANOVA zur Prüfung von Unterschieden zwischen Intensitätslevels
int1_bpm = df[df['intensity'] == 1]['bpm_clean']
int2_bpm = df[df['intensity'] == 2]['bpm_clean']
int3_bpm = df[df['intensity'] == 3]['bpm_clean']

# ANOVA
f_stat_h1, p_value_h1 = f_oneway(int1_bpm, int2_bpm, int3_bpm)

print("=== H1: ANOVA Ergebnisse ===")
print(f"F-Statistik: {f_stat_h1:.3f}")
print(f"p-Wert: {p_value_h1:.6f}")
print(f"Signifikant (α=0.05): {'Ja' if p_value_h1 < 0.05 else 'Nein'}\n")

# Voraussetzungsprüfung: Normalverteilung (Shapiro-Wilk)
print("=== Shapiro-Wilk Test (Normalverteilung) ===")
for intensity in [1, 2, 3]:
    data = df[df['intensity'] == intensity]['bpm_clean'].sample(min(5000, len(df[df['intensity'] == intensity])))
    stat, p = shapiro(data)
    print(f"Intensität {intensity}: W={stat:.4f}, p={p:.4f} - {'Normal' if p > 0.05 else 'Nicht normal'}")

# Varianzhomogenität (Levene-Test)
stat_lev, p_lev = levene(int1_bpm, int2_bpm, int3_bpm)
print(f"\n=== Levene-Test (Varianzhomogenität) ===")
print(f"Levene-Statistik: {stat_lev:.3f}, p={p_lev:.4f} - {'Homogen' if p_lev > 0.05 else 'Nicht homogen'}")

=== H1: ANOVA Ergebnisse ===
F-Statistik: 1332.237
p-Wert: 0.000000
Signifikant (α=0.05): Ja

=== Shapiro-Wilk Test (Normalverteilung) ===
Intensität 1: W=0.9237, p=0.0000 - Nicht normal
Intensität 2: W=0.9186, p=0.0000 - Nicht normal
Intensität 3: W=0.7531, p=0.0000 - Nicht normal

=== Levene-Test (Varianzhomogenität) ===
Levene-Statistik: 8.876, p=0.0001 - Nicht homogen


# H2: Einfluss des Fitnesslevels auf die Herzfrequenz bei verschiedenen Intensitätslevels

## Visualisierungen

In [17]:
# Boxplot nach Fitnesslevel und Intensitätslevel
fig3 = px.box(df, x='fitness_label', y='bpm_clean', color='intensity_label',
              title='Herzfrequenz nach Fitnesslevel und Intensitätslevel',
              labels={'fitness_label': 'Fitnesslevel', 'bpm_clean': 'Herzfrequenz (BPM)', 'intensity_label': 'Intensität'},
              category_orders={'fitness_label': ['niedrig', 'mittel', 'hoch'],
                             'intensity_label': ['niedrig', 'mittel', 'hoch']})
fig3.show()

# Interaktionsdiagramm
mean_data = df.groupby(['fitness_label', 'intensity_label'])['bpm_clean'].mean().reset_index()
fig4 = px.line(mean_data, x='intensity_label', y='bpm_clean', color='fitness_label',
               markers=True,
               title='Interaktionsdiagramm: Fitnesslevel × Intensitätslevel',
               labels={'intensity_label': 'Intensitätslevel', 'bpm_clean': 'Durchschnittliche BPM', 'fitness_label': 'Fitnesslevel'},
               category_orders={'intensity_label': ['niedrig', 'mittel', 'hoch'],
                              'fitness_label': ['niedrig', 'mittel', 'hoch']})
fig4.show()

## Deskriptive Statistik

In [18]:
# Statistik pro Kombination Fitnesslevel × Intensitätslevel
h2_stats = df.groupby(['fitness_label', 'intensity_label'])['bpm_clean'].agg([
    ('Mittelwert', 'mean'),
    ('Standardabweichung', 'std'),
    ('n', 'count')
]).round(2)

print("=== H2: Deskriptive Statistik nach Fitnesslevel und Intensitätslevel ===")
display(h2_stats)

=== H2: Deskriptive Statistik nach Fitnesslevel und Intensitätslevel ===


Mittelwert  Standardabweichung    n
fitness_label intensity_label                                     
mittel        hoch                 175.17               13.71  742
              mittel               158.37               11.62  613
              niedrig              136.21               11.06  614
niedrig       hoch                 175.09               20.87  438
              mittel               164.29               20.83  409
              niedrig              146.07               16.55  362

## Hypothesentests

In [19]:
# Zweifaktorielle ANOVA mit scipy
# Für eine vollständige zweifaktorielle ANOVA verwenden wir scipy's f_oneway für Haupteffekte

print("=== H2: ANOVA Ergebnisse ===\n")

# Haupteffekt: Fitnesslevel
fitness_groups = [group['bpm_clean'].values for name, group in df.groupby('fitness_level')]
f_stat_fitness, p_fitness = f_oneway(*fitness_groups)
print(f"Haupteffekt Fitnesslevel:")
print(f"  F-Statistik: {f_stat_fitness:.3f}, p-Wert: {p_fitness:.6f}")
print(f"  Signifikant: {'Ja' if p_fitness < 0.05 else 'Nein'}\n")

# Haupteffekt: Intensitätslevel
intensity_groups = [group['bpm_clean'].values for name, group in df.groupby('intensity')]
f_stat_intensity, p_intensity = f_oneway(*intensity_groups)
print(f"Haupteffekt Intensitätslevel:")
print(f"  F-Statistik: {f_stat_intensity:.3f}, p-Wert: {p_intensity:.6f}")
print(f"  Signifikant: {'Ja' if p_intensity < 0.05 else 'Nein'}\n")

# Interaktionseffekt: Vergleich der Kombination
interaction_groups = [group['bpm_clean'].values for name, group in df.groupby(['fitness_level', 'intensity'])]
f_stat_interact, p_interact = f_oneway(*interaction_groups)
print(f"Interaktionseffekt (Fitness × Intensität):")
print(f"  F-Statistik: {f_stat_interact:.3f}, p-Wert: {p_interact:.6f}")
print(f"  Signifikant: {'Ja' if p_interact < 0.05 else 'Nein'}\n")

# Voraussetzungsprüfung: Shapiro-Wilk auf Stichproben
print("=== Shapiro-Wilk Test (Normalverteilung) ===")
sample_data = df['bpm_clean'].sample(min(5000, len(df)))
stat_shap, p_shap = shapiro(sample_data)
print(f"Shapiro-Wilk: W={stat_shap:.4f}, p={p_shap:.4f} - {'Normal' if p_shap > 0.05 else 'Nicht normal'}\n")

# Levene-Test für Kombinationen
groups_h2 = [group['bpm_clean'].values for name, group in df.groupby(['fitness_level', 'intensity'])]
stat_lev_h2, p_lev_h2 = levene(*groups_h2)
print(f"=== Levene-Test (Varianzhomogenität) ===")
print(f"Levene-Statistik: {stat_lev_h2:.3f}, p={p_lev_h2:.4f} - {'Homogen' if p_lev_h2 > 0.05 else 'Nicht homogen'}")

=== H2: ANOVA Ergebnisse ===

Haupteffekt Fitnesslevel:
  F-Statistik: 40.552, p-Wert: 0.000000
  Signifikant: Ja

Haupteffekt Intensitätslevel:
  F-Statistik: 1332.237, p-Wert: 0.000000
  Signifikant: Ja

Interaktionseffekt (Fitness × Intensität):
  F-Statistik: 579.390, p-Wert: 0.000000
  Signifikant: Ja

=== Shapiro-Wilk Test (Normalverteilung) ===
Shapiro-Wilk: W=0.9503, p=0.0000 - Nicht normal

=== Levene-Test (Varianzhomogenität) ===
Levene-Statistik: 27.062, p=0.0000 - Nicht homogen


# H3: Einfluss des Raucherstatus auf die Herzfrequenz bei verschiedenen Intensitätslevels

## Visualisierungen

In [20]:
# Violinplot nach Raucherstatus und Intensitätslevel
fig5 = px.violin(df, x='intensity_label', y='bpm_clean', color='smoker_label',
                 title='Herzfrequenz nach Raucherstatus und Intensitätslevel',
                 labels={'intensity_label': 'Intensitätslevel', 'bpm_clean': 'Herzfrequenz (BPM)', 'smoker_label': 'Raucherstatus'},
                 box=True,
                 category_orders={'intensity_label': ['niedrig', 'mittel', 'hoch']})
fig5.show()

# Interaktionsdiagramm
mean_data_h3 = df.groupby(['smoker_label', 'intensity_label'])['bpm_clean'].mean().reset_index()
fig6 = px.line(mean_data_h3, x='intensity_label', y='bpm_clean', color='smoker_label',
               markers=True,
               title='Interaktionsdiagramm: Raucherstatus × Intensitätslevel',
               labels={'intensity_label': 'Intensitätslevel', 'bpm_clean': 'Durchschnittliche BPM', 'smoker_label': 'Raucherstatus'},
               category_orders={'intensity_label': ['niedrig', 'mittel', 'hoch']})
fig6.show()

## Deskriptive Statistik

In [21]:
# Statistik pro Kombination Raucherstatus × Intensitätslevel
h3_stats = df.groupby(['smoker_label', 'intensity_label'])['bpm_clean'].agg([
    ('Mittelwert', 'mean'),
    ('Median', 'median'),
    ('Standardabweichung', 'std'),
    ('n', 'count')
]).round(2)

print("=== H3: Deskriptive Statistik nach Raucherstatus und Intensitätslevel ===")
display(h3_stats)

=== H3: Deskriptive Statistik nach Raucherstatus und Intensitätslevel ===


Mittelwert  Median  Standardabweichung    n
smoker_label intensity_label                                             
Nichtraucher hoch                 175.17  179.10               13.71  742
             mittel               158.37  162.60               11.62  613
             niedrig              136.21  140.19               11.06  614
Raucher      hoch                 175.09  184.62               20.87  438
             mittel               164.29  173.91               20.83  409
             niedrig              146.07  152.67               16.55  362

## Hypothesentests

In [22]:
# Zweifaktorielle ANOVA mit scipy für H3
print("=== H3: ANOVA Ergebnisse ===\n")

# Haupteffekt: Raucherstatus
smoker_groups = [group['bpm_clean'].values for name, group in df.groupby('smoker')]
f_stat_smoker, p_smoker = f_oneway(*smoker_groups)
print(f"Haupteffekt Raucherstatus:")
print(f"  F-Statistik: {f_stat_smoker:.3f}, p-Wert: {p_smoker:.6f}")
print(f"  Signifikant: {'Ja' if p_smoker < 0.05 else 'Nein'}\n")

# Haupteffekt: Intensitätslevel
intensity_groups_h3 = [group['bpm_clean'].values for name, group in df.groupby('intensity')]
f_stat_intensity_h3, p_intensity_h3 = f_oneway(*intensity_groups_h3)
print(f"Haupteffekt Intensitätslevel:")
print(f"  F-Statistik: {f_stat_intensity_h3:.3f}, p-Wert: {p_intensity_h3:.6f}")
print(f"  Signifikant: {'Ja' if p_intensity_h3 < 0.05 else 'Nein'}\n")

# Interaktionseffekt
interaction_groups_h3 = [group['bpm_clean'].values for name, group in df.groupby(['smoker', 'intensity'])]
f_stat_interact_h3, p_interact_h3 = f_oneway(*interaction_groups_h3)
print(f"Interaktionseffekt (Raucher × Intensität):")
print(f"  F-Statistik: {f_stat_interact_h3:.3f}, p-Wert: {p_interact_h3:.6f}")
print(f"  Signifikant: {'Ja' if p_interact_h3 < 0.05 else 'Nein'}\n")

# Voraussetzungsprüfung
print("=== Shapiro-Wilk Test (Normalverteilung) ===")
sample_data_h3 = df['bpm_clean'].sample(min(5000, len(df)))
stat_shap_h3, p_shap_h3 = shapiro(sample_data_h3)
print(f"Shapiro-Wilk: W={stat_shap_h3:.4f}, p={p_shap_h3:.4f} - {'Normal' if p_shap_h3 > 0.05 else 'Nicht normal'}\n")

# Levene-Test
groups_h3 = [group['bpm_clean'].values for name, group in df.groupby(['smoker', 'intensity'])]
stat_lev_h3, p_lev_h3 = levene(*groups_h3)
print(f"=== Levene-Test (Varianzhomogenität) ===")
print(f"Levene-Statistik: {stat_lev_h3:.3f}, p={p_lev_h3:.4f} - {'Homogen' if p_lev_h3 > 0.05 else 'Nicht homogen'}")

=== H3: ANOVA Ergebnisse ===

Haupteffekt Raucherstatus:
  F-Statistik: 40.552, p-Wert: 0.000000
  Signifikant: Ja

Haupteffekt Intensitätslevel:
  F-Statistik: 1332.237, p-Wert: 0.000000
  Signifikant: Ja

Interaktionseffekt (Raucher × Intensität):
  F-Statistik: 579.390, p-Wert: 0.000000
  Signifikant: Ja

=== Shapiro-Wilk Test (Normalverteilung) ===
Shapiro-Wilk: W=0.9503, p=0.0000 - Nicht normal

=== Levene-Test (Varianzhomogenität) ===
Levene-Statistik: 27.062, p=0.0000 - Nicht homogen


# Begründung Statistik

## H1: ANOVA
- Der Test **ANOVA (Analyse der Varianz)** eignet sich für H1, weil überprüft werden soll, ob sich die Herzfrequenz (BPM) zwischen den verschiedenen Intensitätslevels (niedrig, mittel, hoch) signifikant unterscheidet.
- Das Intensitätslevel ist eine kategoriale Variable mit drei Gruppen, während BPM eine kontinuierliche Variable ist.
- Wir prüfen den Zusammenhang zwischen dem Intensitätslevel und der Herzfrequenz.
- Voraussetzungen werden geprüft mit dem **Shapiro-Wilk-Test** für Normalverteilung und dem **Levene-Test** für Varianzhomogenität.

## H2: Zweifaktorielle ANOVA
- Der Test **zweifaktorielle ANOVA** eignet sich für H2, weil sowohl das Fitnesslevel als auch das Intensitätslevel kategoriale unabhängige Variablen sind.
- Wir prüfen den Einfluss des Fitnesslevels und des Intensitätslevels auf die Herzfrequenz sowie mögliche Wechselwirkungen.
- Voraussetzungen werden geprüft mit dem **Shapiro-Wilk-Test** für Normalverteilung der Residuen und dem **Levene-Test** für Varianzhomogenität.

## H3: Zweifaktorielle ANOVA
- Der Test **zweifaktorielle ANOVA** eignet sich für H3, weil sowohl der Raucherstatus als auch das Intensitätslevel kategoriale unabhängige Variablen sind.
- Wir prüfen den Einfluss des Raucherstatus und des Intensitätslevels auf die Herzfrequenz sowie mögliche Wechselwirkungen.
- Voraussetzungen werden geprüft mit dem **Shapiro-Wilk-Test** und **Levene-Test**.

# Erwartete Ergebnisse

**H1:** Wir erwarten, dass mit steigendem Intensitätslevel auch die Herzfrequenz (BPM) zunimmt, da höhere Belastung zu stärkerer kardiovaskulärer Aktivität führt.

**H2:** Zudem wird erwartet, dass Personen mit höherem Fitnesslevel bei gleicher Intensität eine niedrigere Herzfrequenz aufweisen, da trainierte Personen effizienter auf Belastung reagieren.

**H3:** Weiter erwarten wir, dass Raucher im Vergleich zu Nichtrauchern eine höhere Herzfrequenz zeigen, insbesondere bei höheren Intensitätslevels, da Rauchen die Sauerstoffaufnahme und Herzleistung beeinträchtigt.

**Mögliche Einflussfaktoren:**
- **Alter:** Ältere Personen haben tendenziell eine geringere maximale Herzfrequenz
- **Geschlecht:** Männer und Frauen zeigen unterschiedliche kardiovaskuläre Reaktionen
- **Fitnesslevel:** Beeinflusst die Anpassung des Herz-Kreislauf-Systems
- **Übungstyp:** Verschiedene Bewegungsformen verursachen variierende Belastungen
- **Schlaf und Ernährung:** Können die Herzfrequenz kurzfristig beeinflussen